In [ ]:
%matplotlib widget

In [ ]:
import numpy as np
import tifffile as tiff
from pathlib import Path
from matplotlib import pyplot as plt
import flammkuchen as fl
import napari
from napari_animation import AnimationWidget
from split_dataset import SplitDataset
from fimpylab.core.lightsheet_experiment import LightsheetExperiment
%gui qt5

In [ ]:
master = Path(r"Z:\Hagar\crazy fish\e0040_v10")

fish = Path(r"Z:\Hagar\crazy fish\230302_f0\e0040_v10")

In [ ]:
# Getting stimulus information
exp = LightsheetExperiment(path=fish)
stim_log = getattr(exp, "stimulus_log")
fs = exp.fn

stim_regressors = fl.load(fish / "sensory_regressors.h5")
stim_t = stim_log.t.values

In [ ]:
stim_regressors.keys()

In [ ]:
individual_theta = stim_regressors['individual_theta_interp']
np.shape(individual_theta)

In [ ]:
# Loading imaging data (aligned)
aligned_stack = SplitDataset(fish / "aligned")
len_rec_original, num_planes, x_pix, y_pix = np.shape(aligned_stack)
np.shape(aligned_stack)

n_trials = np.shape(np.where(np.diff(individual_theta[2]) > 0))[1] // 2

print(n_trials)

In [ ]:
triggered_data_timing = np.zeros((8, n_trials), dtype=int)
for i in range(8):
    temp_theta = individual_theta[i]
    diff_theta = np.where(np.diff(temp_theta) > 0)[0][::2]
    print(diff_theta)
    triggered_data_timing[i,:] = diff_theta

In [ ]:
#triggered_data = np.zeros((8, int(15*fs), num_planes, x_pix, y_pix)) 
for direction in range(5, 8):
    print("direction: ", direction)
    for trial in range(n_trials):
        t_start = int(triggered_data_timing[direction,trial])
        t_end = int(t_start + (15*fs))
        print("t_start: ", t_start)
        triggered_data[direction,:,:,:] += aligned_stack[t_start:t_end]

In [ ]:
# Generating videos of trial triggered activity:
viewer = napari.Viewer()
for i in range(8):
    viewer.add_image(triggered_data[i,:,:,:], colormap="gray_r")



In [ ]:
animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area='right')

In [ ]:
# Generating videos of trial triggered activity (planes):
viewer = napari.Viewer()
for i in range(8):
    stack = forward_trials_red[i,:,:,:]
    viewer.add_image(stack, colormap="gray_r")
animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area='right')